### TO-DO

- 무신사 홈페이지에서 랭킹 상품들에 대한 정보를 가옴

In [ ]:
# 라이브러리 정의
import pandas as pd
from selenium.webdriver import Chrome
import time
from tqdm.notebook import tqdm
import warnings
warnings.simplefilter('ignore')

In [ ]:
search_page = 1         #검색할 페이지수

brand_names = []        #브랜드명 
product_names = []      #제품명
prices = []             #가격
review_counts = []      #리뷰
favorites = []          #좋아요
coupons = []            #쿠폰
specials = []           #특


driver = Chrome('./chromedriver/chromedriver')

for page_no in tqdm(range(search_page)):

    #무신사 랭킹페이지 열기
    page_url = f"https://www.musinsa.com/ranking/best?period=now&age=ALL&mainCategory=&subCategory=&leafCategory=&price=&golf=false&kids=false&newProduct=false&exclusive=false&discount=false&soldOut=false&page={page_no+1}&viewType=small&priceMin=&priceMax="
    driver.get(page_url)
    time.sleep(3)



    # 스크롤 높이 가져옴
    last_height = driver.execute_script("return document.body.scrollHeight")

    while True:
        # 끝까지 스크롤 다운
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")

        # 3초 대기
        time.sleep(3)

        # 스크롤 다운 후 스크롤 높이 다시 가져옴
        new_height = driver.execute_script("return document.body.scrollHeight")
        if new_height == last_height:
            break
        last_height = new_height

    list_basis = driver.find_element_by_id("goodsRankList")
    item_list = list_basis.find_elements_by_class_name('li_box')

    #print(list_basis)
    #print(item_list)

    items = []
    for i in tqdm(range(len(item_list))):
        item = item_list[i]

        #t = item.find_element_by_class_name('article_info')
        #print(t.text)
        
        #브랜드명 가져기
        try:
            brand_name = item.find_element_by_class_name('item_title')
            brand_names.append(brand_name.text)
            #print(brand_name.text)
        except Exception as error:
            print(error)

        #제품명 가져기
        try:
            product_name = item.find_element_by_class_name('list_info')
            product_names.append(product_name.text)
            #print(product_name.text)
        except Exception as error:
            print(error)

        
        #가격 가져오기
        try:
            price = item.find_element_by_class_name("price")#.text[:-1].replace(',', '')
            price = price.text.split("\n")[-1]
            price = price[:-1].replace(',', '')
            prices.append(int(price))
            #print(price)
        except Exception as error:
            print(error)

        #쿠폰 가져오기
        try:
            coupon = item.find_element_by_class_name("txt_discount_price").text[:-1].replace(',', '')
            coupons.append(coupon)
            #print(coupon)
        except Exception as error:
            print(error)
            coupons.append(0)
            #print(0)

        #리뷰 가져기
        try:
            review_count = item.find_element_by_class_name("point").text.replace(',', '')
            #review_count = review_count.text.replace(',', '')
            review_counts.append(int(review_count))
            #print(review_count)
        except Exception as error:
            print(error)
            review_counts.append(0)
            #print(0)

        #좋아요 가져기
        try:
            favorite = item.find_element_by_name("count").text.replace(',', '')
            #favorite = favorite.text.replace(',', '')
            favorites.append(int(favorite))
            #print(favorite)
        except Exception as error:
            print(error)
            favorites.append(0)
            #print(0)

        #특 가져오기
        try:
            special = item.find_element_by_class_name("box-icon-right")
            specials.append(special.text)
            #print(special.text) 
        except Exception as error:
            print(error)
            specials.append("")
            #print("") 



가져온 리스트 개수확인

In [ ]:
print(len(brand_names), len(product_names), len(prices), len(coupons), len(review_counts),  len(favorites),  len(specials))

리스트 데이터프레임으로 저장

In [ ]:
result = pd.DataFrame({
                        "브랜드명" : brand_names,
                        "제품명" : product_names,
                        "가격" : prices,
                        "쿠폰" : coupons,
                        "리뷰수" : review_counts,
                        "좋아요" : favorites,
                        "특" : specials
                        })
result

엑셀로 저장시키기

In [ ]:
result.to_excel(f"./data/musinsa_ranklist({search_page}page)_{time.strftime('%Y-%m-%d-%I', time.localtime(time.time()))}.xlsx", index=False)